In [ ]:
from ase.db import connect
import numpy as np
import xarray as xr

In [ ]:
db = connect('/nc/SHNITSEL-data/CH2NH2.db')

In [ ]:
row0 = next(db.select())

In [ ]:
dir(row0)

In [ ]:
row0.data

In [ ]:
keys = list(row0.data)
keys

In [ ]:
# Assuming dimensions follow SchNarc order
shapes = {
    'energy': ['frame', 'state'],
    'socs': ['frame', 'soc'],
    'forces': ['frame', 'state', 'atom', 'direction'],
    'nacs': ['frame', 'statecomb', 'atom', 'direction'],
    'dipoles': ['frame', 'not_sure', 'direction'],
    # TODO Not sure what the second dipole dimension means.
    # 3 permanent dipoles + 3 transition dipoles?
}

In [ ]:
def stack_rows(name):
    global db
    return np.stack([row.data[name] for row in db.select()])

energy = stack_rows('energy')
energy

In [ ]:
data_vars = {
    name: (dims, stack_rows(name))
    for name, dims in shapes.items()
}
data_vars

In [ ]:
frames = xr.Dataset(data_vars)
frames

In [ ]:
# In case I was right about the dipoles
dipoles = data_vars['dipoles'][1]
dip_perm = dipoles[:,:3,:]
dip_trans = dipoles[:,3:,:]
del(data_vars['dipoles'])

data_vars['dip_perm'] = (['frame', 'state', 'direction'], dip_perm)
data_vars['dip_trans'] = (['frame', 'statecomb', 'direction'], dip_trans)

In [ ]:
frames = xr.Dataset(data_vars)
frames

In [ ]:
frames.to_netcdf('/tmp/output.nc', engine='h5netcdf')